In [1]:
import toml
import json
from scipy.constants import R
from moleculegraph.general_utils import read_pair_potentials, read_bond_potentials, read_angle_potentials, read_torsion_potentials

# Write force field toml files #

This notebook is used to translate the TAMie force field files into a moleculegraph compatible toml file. The units are also converted to LAMMPS standard units (kcal/mol).

### Note:
In TOML one cannot mix integers and floats in one list! The nharmonic style needs as first parameter the length of the expansion, hence an integer.
Therefore, it is better to save the force field for alcohols as json file. pyLAMMPS can read both json or toml.

In [5]:
# Define the TAMie force field folder
path_to_tamie    = "force-fields/UA_TAMie_alcohols/"

# Define the output toml file
output_file_name = "force-fields/ff_UA_TAMie_alcohols.toml"

# Read in the force field files using moleculegraph

pair_pots    = read_pair_potentials( path_to_tamie + "pair_potentials" )
bond_pots    = read_bond_potentials( path_to_tamie + "bond_potentials" )  
angle_pots   = read_angle_potentials( path_to_tamie + "angle_potentials" )
torsion_pots = read_torsion_potentials( path_to_tamie + "torsion_potentials" )

# Converstion factor from unit Kelvin to kcal/mol (K * J/(K*mol) * kcal/J)
KB_kcal = R / 4184

# Define pair interactions (sigma in Angstrom, epsilon in kcal/mol)
for p in pair_pots:
    pair_pots[p]["vdw_style"] = "mie/cut"
    pair_pots[p]["coulomb_style"] = "" if pair_pots[p]["charge"] == 0.0 else "coul/long" 
    pair_pots[p]["epsilon"] = round( pair_pots[p]["epsilon"] * KB_kcal, 4 )
    pair_pots[p]["n"] = pair_pots[p]["m"]
    pair_pots[p]["m"] = 6

# Define bonded interactions (Spring constant in kcal/mol / Angstrom^2. As TAMie utilize fixed bond, use empirical value: https://doi.org/10.1021/acs.jced.9b01031 )
for p in bond_pots:
    bond_pots[p]["style"] = "harmonic"
    bond_pots[p]["p"] = [ 200.0, bond_pots[p]["p"][0] ]

# Define angle interactions (Spring constant in kcal/mol / rad^2) --> introduce the factor 1/2 in the constant (see LAMMPS documentary)
for p in angle_pots:
    angle_pots[p]["style"] = "harmonic"
    angle_pots[p]["p"] = [ round(angle_pots[p]["p"][1] * KB_kcal/2, 4 ), angle_pots[p]["p"][0] ]

# Define dihedral interactions (Spring constant in kcal/mol)
# Leave out last value from the torsion potdentials, as its always 0.0.
for p in torsion_pots:
    if torsion_pots[p]["type"] == 1:
        torsion_pots[p]["style"]     = "opls"
        # LAMMPS opls uses 1/2*K, but TAMie uses K, hence multiply with two.
        # TAMie utilizes a C0 constant and end at C3 while LAMMPS starts with C1 and ends with C4, hence leave out first entry and add an 0.0 for C4
        torsion_pots[p]["p"]         = [ round( x * KB_kcal * 2, 4 ) for x in torsion_pots[p]["p"][1:-1] ] + [ 0.0 ]

    elif torsion_pots[p]["type"] == 6:
        torsion_pots[p]["style"]     = "nharmonic"
        # No prefactor is used. Add number of parameters of the nharmonic 
        torsion_pots[p]["p"]         = [ len(torsion_pots[p]["p"][:-1]) ] + [ round( x * KB_kcal, 4 ) for x in torsion_pots[p]["p"][:-1] ]
    

# Write toml file
forcefield = {}
forcefield["atoms"]    = pair_pots
forcefield["bonds"]    = bond_pots
forcefield["angles"]   = angle_pots
forcefield["torsions"] = torsion_pots





#with open(output_file_name, "w") as toml_file:
#    toml.dump(forcefield, toml_file)


with open(output_file_name.replace("toml","json"), "w") as toml_file:
    json.dump(forcefield,toml_file,indent=1)